# Geolocalización con MultiProcessing + IV Palabras

En esta notebook haremos un intento de geolocalización con los textos de los usuarios...

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd

df_train = pd.read_json("../data/geoloc/users_train.json")
df_test = pd.read_json("../data/geoloc/users_test.json")

Hagamos lo siguiente:

- Entrenemos con unigramas una regresión logística para 
- Luego probemos con los regionalismos

Primero, partamos en train, test

In [2]:
df_train.groupby("provincia").count()

,text
provincia,
buenosaires,337
catamarca,341
chaco,331
chubut,328
cordoba,317
corrientes,345
entrerios,338
formosa,286
jujuy,339


## Palabras precalculadas

Carguemos antes las palabras que sabemos que ocurren una cantidad razonable de veces

In [3]:
%%time
from contrastes.processing import build_dataframe_from_users
from contrastes.processing import preprocess_raw_df


#word_df = build_dataframe_from_users(row for index, row in df_train.iterrows())

word_df = pd.read_csv("train_word_df_filtered.csv", index_col=0)
word_df = preprocess_raw_df(word_df, filter_words=(10, 2))

CPU times: user 1.52 s, sys: 188 ms, total: 1.71 s
Wall time: 1.71 s


/home/jmperez/projects/contrastes/contrastes/processing.py:185: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_palabras = cant_palabras
/home/jmperez/projects/contrastes/contrastes/processing.py:186: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_personas = cant_personas


La reg. logística será un softmax, así que elijo `multi_class='multinomial'`

In [4]:
from contrastes.lists import add_ival

add_ival(word_df, normalize=True)

Calculating information values...
Calculating ranks...


In [5]:
word_df.sort_values("rank_palabras", ascending=True, inplace=True)

word_df.iloc[:10]

,buenosaires_ocurrencias,buenosaires_usuarios,catamarca_ocurrencias,catamarca_usuarios,chaco_ocurrencias,chaco_usuarios,chubut_ocurrencias,chubut_usuarios,cordoba_ocurrencias,cordoba_usuarios,...,tucuman_usuarios,cant_provincias,cant_palabra,cant_usuarios,ival_palabras,ival_personas,ival_palper,rank_palabras,rank_personas,rank_palper
hoa,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6,3281.0,16.0,1.673924,0.394085,0.659668,1.0,12428.0,595.0
rioja,21.0,8.0,219.0,73.0,3.0,3.0,18.0,11.0,23.0,14.0,...,16.0,23,8020.0,1012.0,1.658451,0.843488,1.398883,2.0,965.0,50.0
ushuaia,19.0,5.0,2.0,2.0,3.0,2.0,16.0,13.0,3.0,3.0,...,2.0,23,5874.0,704.0,1.644963,1.052576,1.731448,3.0,288.0,14.0
chivilcoy,2331.0,125.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,6,2337.0,262.0,1.599973,1.569203,2.510682,4.0,5.0,1.0
bragado,1757.0,89.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,5,1766.0,192.0,1.535212,1.443644,2.216299,5.0,14.0,2.0
jujuy,17.0,8.0,53.0,23.0,20.0,18.0,22.0,16.0,50.0,21.0,...,49.0,23,8334.0,1464.0,1.496421,0.591684,0.885409,6.0,3309.0,253.0
tilly,0.0,0.0,0.0,0.0,0.0,0.0,1409.0,79.0,1.0,1.0,...,0.0,6,1423.0,180.0,1.473581,1.324791,1.952187,7.0,48.0,5.0
tdf,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,11,1399.0,178.0,1.442562,1.157514,1.669785,8.0,143.0,19.0
rada,0.0,0.0,1.0,1.0,1.0,1.0,1914.0,169.0,7.0,3.0,...,1.0,18,1988.0,424.0,1.437460,1.236516,1.777443,9.0,84.0,11.0
gestionando,6.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,3.0,2.0,...,2.0,20,1646.0,80.0,1.418463,0.157763,0.223781,10.0,67431.0,4702.0


Veamos qué performance tiene usando 1000, 2000, 3000, y así...

In [6]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from contrastes.text import tokenize

liw_vectorizer = CountVectorizer(
    tokenizer=tokenize,
    vocabulary=word_df.index)

X_train = liw_vectorizer.fit_transform(df_train["text"])
print("Vectorizing")
X_test = liw_vectorizer.transform(df_test["text"])

Vectorizing
CPU times: user 10min 15s, sys: 1.05 s, total: 10min 16s
Wall time: 10min 16s


Ya las tenemos vectorizadas en el orden esperado!

In [7]:
from sklearn.preprocessing import LabelEncoder

province_encoder = LabelEncoder()

province_encoder.fit(df_train["provincia"].values)

y_train = province_encoder.transform(df_train["provincia"].values)
y_test = province_encoder.transform(df_test["provincia"].values)

In [8]:
%%time
from contrastes.classifiers import fit_classifiers

num_words_to_fit = list(range(250, 5000, 250))

ret = fit_classifiers(X_train, y_train, X_test, y_test, province_encoder=province_encoder,
                      range_num_words=num_words_to_fit, num_jobs=8)

Entrenando con 250 palabras
Entrenando con 500 palabras
Entrenando con 750 palabras
Entrenando con 1500 palabras
Entrenando con 1000 palabras
Entrenando con 1750 palabras
Entrenando con 1250 palabras
Entrenando con 2000 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


250   palabras ----> accuracy 59.68 mean distance 303.4612
Entrenando con 2250 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


500   palabras ----> accuracy 61.84 mean distance 354.1188
Entrenando con 2500 palabras


...

1500  palabras ----> accuracy 63.08 mean distance 375.3432
Entrenando con 3500 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


1750  palabras ----> accuracy 65.52 mean distance 347.0192
Entrenando con 3750 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


2000  palabras ----> accuracy 65.88 mean distance 344.8376
Entrenando con 4000 palabras
2250  palabras ----> accuracy 65.64 mean distance 349.466
Entrenando con 4250 palabras
2500  palabras ----> accuracy 65.44 mean distance 339.9712
Entrenando con 4500 palabras
2750  palabras ----> accuracy 65.56 mean distance 338.302
Entrenando con 4750 palabras
3000  palabras ----> accuracy 66.08 mean distance 331.96
3500  palabras ----> accuracy 67.00 mean distance 320.8644
3250  palabras ----> accuracy 66.72 mean distance 326.204
3750  palabras ----> accuracy 66.80 mean distance 307.7488
4000  palabras ----> accuracy 66.80 mean distance 307.2376
4250  palabras ----> accuracy 66.88 mean distance 299.902
4500  palabras ----> accuracy 67.32 mean distance 297.5956
4750  palabras ----> accuracy 67.40 mean distance 295.1436
CPU times: user 64 ms, sys: 488 ms, total: 552 ms
Wall time: 20.2 s


In [9]:
%%time
from contrastes.classifiers import fit_classifiers

num_words_to_fit = list(range(5000, 20000, 500))

ret += fit_classifiers(X_train, y_train, X_test, y_test, province_encoder=province_encoder,
                      range_num_words=num_words_to_fit, num_jobs=8)

Entrenando con 8000 palabras
Entrenando con 6000 palabras
Entrenando con 7000 palabras
Entrenando con 6500 palabras
Entrenando con 5500 palabras
Entrenando con 7500 palabras
Entrenando con 8500 palabras
Entrenando con 5000 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


5000  palabras ----> accuracy 67.32 mean distance 296.62
Entrenando con 9000 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


5500  palabras ----> accuracy 67.60 mean distance 296.6752
Entrenando con 9500 palabras


...

7500  palabras ----> accuracy 67.64 mean distance 299.4144
Entrenando con 11500 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


8000  palabras ----> accuracy 67.32 mean distance 298.2068
Entrenando con 12000 palabras


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


8500  palabras ----> accuracy 67.36 mean distance 298.2964
Entrenando con 12500 palabras
9000  palabras ----> accuracy 67.80 mean distance 295.5116
Entrenando con 13000 palabras
9500  palabras ----> accuracy 65.48 mean distance 313.7744
Entrenando con 13500 palabras
10000 palabras ----> accuracy 65.84 mean distance 308.8908
Entrenando con 14000 palabras
10500 palabras ----> accuracy 65.92 mean distance 309.2316
Entrenando con 14500 palabras
11000 palabras ----> accuracy 66.72 mean distance 297.5412
Entrenando con 15000 palabras
11500 palabras ----> accuracy 66.60 mean distance 298.2156
Entrenando con 15500 palabras
12000 palabras ----> accuracy 66.36 mean distance 304.4724
Entrenando con 16000 palabras
12500 palabras ----> accuracy 66.72 mean distance 302.4672
Entrenando con 16500 palabras
13000 palabras ----> accuracy 66.92 mean distance 300.2012
Entrenando con 17000 palabras
13500 palabras ----> accuracy 66.92 mean distance 300.2784
Entrenando con 17500 palabras
14000 palabras ----> 

In [10]:
for r in ret:
    num_words = r["num_words"]
    acc = r["accuracy"]
    md = r["mean_distance"]
    print("{:<5} palabras ----> accuracy {:.2f} mean distance {}".format(
        num_words, acc*100, md
    ))

250   palabras ----> accuracy 59.68 mean distance 303.4612
500   palabras ----> accuracy 61.84 mean distance 354.1188
750   palabras ----> accuracy 63.92 mean distance 340.1252
1000  palabras ----> accuracy 62.68 mean distance 358.928
1250  palabras ----> accuracy 62.20 mean distance 386.8308
1500  palabras ----> accuracy 63.08 mean distance 375.3432
1750  palabras ----> accuracy 65.52 mean distance 347.0192
2000  palabras ----> accuracy 65.88 mean distance 344.8376
2250  palabras ----> accuracy 65.64 mean distance 349.466
2500  palabras ----> accuracy 65.44 mean distance 339.9712
2750  palabras ----> accuracy 65.56 mean distance 338.302
3000  palabras ----> accuracy 66.08 mean distance 331.96
3250  palabras ----> accuracy 66.72 mean distance 326.204
3500  palabras ----> accuracy 67.00 mean distance 320.8644
3750  palabras ----> accuracy 66.80 mean distance 307.7488
4000  palabras ----> accuracy 66.80 mean distance 307.2376
4250  palabras ----> accuracy 66.88 mean distance 299.902
4500

In [13]:
import pickle

pickle.dump(ret, open("res_iv_palabras.pkl", "wb"))

In [20]:
new_ret = pickle.load(open("res_iv_palabras.pkl", "rb"))

clf = new_ret[1]["clf"]

clf.coef_.shape

(23, 500)